<h1 style="text-align:center">Héritage CAE</h1> 

Ce Notebook est un travail préparatif au développement d'un Simulateur de réforme de l'héritage basé sur les [travaux du CAE-ECO](https://www.cae-eco.fr/repenser-lheritage) 

In [1]:
from IPython.display import display
%matplotlib widget
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Import des données

## Héritage net par quantile

Sources : extraction "visuelle" des graphiques [33 & 34 du Focus](doc/cae-focus077-2021.pdf)

In [2]:
# Read inheritance data
df_herit = pd.read_csv("data/heritage.csv", comment="#")
df_herit = df_herit.rename(columns={key: key.split("[")[0] for key in df_herit.columns})
df_herit.quant = df_herit.quant
df_herit = df_herit.set_index("quant")
df_herit

,heritage
quant,
10.0,0
20.0,15000
30.0,47000
40.0,65000
50.0,65000
60.0,65000
70.0,84000
80.0,164000
90.0,288000


## Taux effectif d'imposition pour chaque scenario

Source: extraction visuelle du graphique 28 du [Focus](doc/cae-focus077-2021.pdf), page 31. 

In [3]:
# Read taux effectif
df_taux = pd.read_csv("data/taux_effectifs.csv", comment="#")
df_taux = df_taux.rename(columns={key: key.split("[")[0] for key in df_taux.columns})
df_taux.quant = df_taux.quant
df_taux = df_taux.set_index("quant")
df_taux

SCENARIOS = list(df_taux.columns)

print("Scenarios", SCENARIOS)

# Jointure des deux ensembles de données
df = df_herit.join(df_taux)
df

Scenarios ['actuel', 'assiette1-bareme1', 'assiette2-bareme2', 'assiette1-bareme3', 'assiette2-bareme4']


,heritage,actuel,assiette1-bareme1,assiette2-bareme2,assiette1-bareme3,assiette2-bareme4
quant,,,,,,
10.0,0,0.0,0.0,0.0,0.0,0.0
20.0,15000,0.0,0.0,0.0,0.0,0.0
30.0,47000,0.0,0.0,0.0,0.0,0.0
40.0,65000,0.0,0.0,0.0,0.0,0.0
50.0,65000,0.0,0.0,0.0,0.0,0.0
60.0,65000,0.0,0.0,0.0,0.0,0.0
70.0,84000,0.0,1.0,1.0,0.0,0.0
80.0,164000,1.0,1.0,2.0,0.0,0.0
90.0,288000,2.0,2.0,5.0,1.0,1.0


# Calculs 

## Volumes

On calcule les volumes moyen d'héritage pour chaque quantile.

In [5]:
# Compute volumes for each quantile
quantiles = np.insert(df.index.values, 0, [0])
df["volumes"]  = df.heritage.values * (quantiles[1:] - quantiles[0:-1]) / 100

display(df)

average = df.volumes.sum()
print("Héritage net moyen", df.volumes.sum())

base = 300*10**9 / average

print("Base calculée (sur hypothèse flux de succession 300 Mds€):", base)

,heritage,actuel,assiette1-bareme1,assiette2-bareme2,assiette1-bareme3,assiette2-bareme4,volumes
quant,,,,,,,
10.0,0,0.0,0.0,0.0,0.0,0.0,0.000
20.0,15000,0.0,0.0,0.0,0.0,0.0,1500.000
30.0,47000,0.0,0.0,0.0,0.0,0.0,4700.000
40.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000
50.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000
60.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000
70.0,84000,0.0,1.0,1.0,0.0,0.0,8400.000
80.0,164000,1.0,1.0,2.0,0.0,0.0,16400.000
90.0,288000,2.0,2.0,5.0,1.0,1.0,28800.000


Héritage net moyen 190502.1119999995
Base calculée (sur hypothèse flux de succession 300 Mds€): 1574785.6905649465


### Quelle est la base de la distribution ?

<div style="background:yellow">
⚠ Question : Quelle est la "base" de ces quantiles ?

On trouve un héritage net moyen de **190k€.**
Rapporté à un flux de 300 Md € [page 5 de la Note](./doc/cae-note069s.pdf), ça ferait une base de <b>1.5 millions de ??<> héritages / de décès / d'héritiers ? Ca semble loin des 720k naissances / 667k décès annuels.
</div>

## Recettes fiscales

Calcul des héritages bruts, avant imposition actuelle.
Puis calcul des recettes fiscales pour chaque scenario.

In [6]:
df["volumes_brut"] = df.volumes * 100 / (100 - df.actuel)
df["heritage_brut"] = df.heritage * 100 / (100 - df.actuel)

recettes = dict()
for scenar in SCENARIOS :
    total = (df.volumes_brut *  df[scenar] / 100).sum()
    recettes[scenar] = total
    
recettes = pd.DataFrame(
    recettes.items(), columns=["scenario", "recettes_moyennes"]).set_index("scenario")

# Différence de recette par rapport au scenario actuel
recettes["recettes_diff"] = recettes.recettes_moyennes - recettes.recettes_moyennes[0]
recettes

,recettes_moyennes,recettes_diff
scenario,,
actuel,8417.161523,0.000000
assiette1-bareme1,15396.690339,6979.528816
assiette2-bareme2,22521.564814,14104.403290
assiette1-bareme3,8451.240735,34.079211
assiette2-bareme4,17693.026402,9275.864879


### Quelle base ?

A nouveau se pose la question de la base.
Si on reprend la base précedente empirique 1.5 millions, on obtient :

In [7]:
# Recettes moyennes avec une  base 1.5 millions 
pd.merge(
    recettes.recettes_moyennes * base,
    recettes.recettes_diff * base, left_index=True, right_index=True)

,recettes_moyennes,recettes_diff
scenario,,
actuel,1.325523e+10,0.000000e+00
assiette1-bareme1,2.424649e+10,1.099126e+10
assiette2-bareme2,3.546664e+10,2.221141e+10
assiette1-bareme3,1.330889e+10,5.366745e+07
assiette2-bareme4,2.786272e+10,1.460750e+10


On retombe sur des recettes fiscales actuelles de **13 Mds €** :du même ordre de grandeur que **15 Mds €** (2022)
Par contre les recette supplémentaires sont un peu au delà de celles notées dans le [graphique 28 du focus](./doc/cae-focus077-2021.pdf) (page 31). Respectivement de 9, 19, 0 et 12 Mds€.

# Calcul héritage de base

On fait ici un calcul simple de l'héritage de base financable par un surplus donné.
On cherche combien de quantiles on peut compléter à un héritage X, avec un surplus S
Tant que le l'héritage min calculé dépasse l"héritage de la tranche suivante, on élargit la redistribution au suivant.

In [47]:
def heritage_de_base(surplus, base=1) :
  
    quant = np.insert(df.index.values, 0, [0])
  
    # Taille chaque segment 
    # XXX hardcodé à 0.1 (premier quantiles considéres uniquement)
    width=0.1
    
    heritages = df.heritage.values
    
    # Boucle sur les quantiles : tant que l'héritage min déborde sur le suivant 
    for i in range(len(heritages)) :
        
        if (quant[i+1] - quant[i])/100 != width :
            raise Exception("Pas d'héritage minimum trouvé pour les premiers quantiles")
        
        heritage_min = (surplus / (width*base) + np.sum(heritages[0:i]))/(i+1)
        
        print("Quantile %d%%-%d%%. Héritage min :%d" % (quant[i+1], quant[i], heritage_min))
        
        if heritage_min < heritages[i+i]:
            break
        else:
            print("> %d => continue" % heritages[i+1])
            
    return heritage_min

In [29]:
heritage_de_base(13*10**9)

Quantile 10%-0%. Héritage min :82550
> 15000 => continue
Quantile 20%-10%. Héritage min :41275


41275.45759999989

In [35]:
df

,heritage,actuel,assiette1-bareme1,assiette2-bareme2,assiette1-bareme3,assiette2-bareme4,volumes,heritage_brut,volumes_brut
quant,,,,,,,,,
10.0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000e+00,0.000000
20.0,15000,0.0,0.0,0.0,0.0,0.0,1500.000,1.500000e+04,1500.000000
30.0,47000,0.0,0.0,0.0,0.0,0.0,4700.000,4.700000e+04,4700.000000
40.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000,6.500000e+04,6500.000000
50.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000,6.500000e+04,6500.000000
60.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000,6.500000e+04,6500.000000
70.0,84000,0.0,1.0,1.0,0.0,0.0,8400.000,8.400000e+04,8400.000000
80.0,164000,1.0,1.0,2.0,0.0,0.0,16400.000,1.656566e+05,16565.656566
90.0,288000,2.0,2.0,5.0,1.0,1.0,28800.000,2.938776e+05,29387.755102


In [105]:
def compare(df, scenario) :
    
    df = df.copy()
    
    # Calcul des volumes brut et héritages bruts
    df["volumes_brut"] = df.volumes * 100 / (100 - df.actuel)
    df["heritage_brut"] = df.heritage * 100 / (100 - df.actuel)

    # Calcul du surplus
    surplus = np.sum(df["volumes_brut"] * (df[scenario] - df.actuel)/ 100)
    
    print("Surplus moyen", surplus)
    
    heritage_min = heritage_de_base(surplus)
    
    df["nouveau_net"] = df["heritage_brut"] * (1-df[scenario]/100)
    
    df["nouveau_net"] = np.maximum(df["nouveau_net"], heritage_min)
    
    quantiles = np.insert(df.index.values, 0, [0])
    quantiles_str = []
    for i, quant in enumerate(quantiles) :
        if i >= len(df) :
            break
        if quant < 99 :
            quant2 = quantiles[i+1]
            quantiles_str.append("%d-%d %%" % (quant, quant2))
        else:
            quantiles_str.append(">%0.1f %%" % (100-quant)) 
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=quantiles_str,
            y=[heritage_min]*len(quantiles),
            name='Héritage de base',
            line_color='red',
            mode='lines',
            line_dash = 'dash'
            ))
    
    fig.add_trace(go.Bar(x=quantiles_str,
                y=df.heritage,
                name='Avant réforme',
                marker_color='rgb(55, 83, 109)'
                ))
    fig.add_trace(go.Bar(x=quantiles_str,
                y=df.nouveau_net,
                name='Après réforme',
                marker_color='rgb(26, 118, 255)'
                ))
    
    fig.update_layout(
        title='Héritage avant et après réforme de la fiscalité, pour chaque tranche',
        xaxis_tickfont_size=14,
        xaxis=dict(
            title='Part de la population',
            tickangle=-45),
        yaxis=dict(
            type="log",
            title='Héritage net € (log)',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        #bargap=0.15, # gap between bars of adjacent location coordinates.
        #bargroupgap=0.1 # gap between bars of the same location coordinate.
    )
    fig.show()

In [106]:
compare(df, "assiette2-bareme2")

Surplus moyen 14104.40329035873
Quantile 10%-0%. Héritage min :141044
> 15000 => continue
Quantile 20%-10%. Héritage min :70522
> 47000 => continue
Quantile 30%-20%. Héritage min :52014
